# Singular Representation

Singleton… or borg?

To properly implement a singleton in Python requires either a factory `classmethod` be utilized, or a metaclass which can intervene in the process of constructing a new instance of the class it is used to describe.

Each suffers from its own problems; multiple metaclasses can not contribute towards the construction of a single class, which can be restrictive. Factory methods, esp. mandatory factory methods, unnessicarily make code more verbose.

## The Factory Tactic

In [13]:
class LazySingleton:
    _instance = None  # We use a class attribute to store the singleton.
    
    @classmethod
    def new(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = cls(*args, **kw)
        
        return cls._instance

In [14]:
a = LazySingleton()
b = LazySingleton()

a is b

False

In [15]:
a = LazySingleton.new()
b = LazySingleton.new()

a is b

True

## Metaclass Hill

Use of a metaclass, while restrictive, is also far more elegant by seamlessly integrating into the standard object construction and instantiation process.

Original [source](http://www.garyrobinson.net/2004/03/python_singleto.html).

In [16]:
class Singleton(type):
    def __init__(cls, name, bases, scope):
        super(Singleton, cls).__init__(name, bases, scope)
        cls.instance = None

    def __call__(cls, *args, **kw):
        if cls.instance is None:
            cls.instance = super(Singleton, cls).__call__(*args, **kw)

        return cls.instance

In [17]:
class Singular(metaclass=Singleton):
    ...

In [18]:
a = Singular()
b = Singular()

a is b

True

## I am Popeye of Borg

You will be askimilgrated.

A more Pythonic approach to "singletons" is to produce the desired *effect* rather than following the exact implementation, which may favour the design of another language. I'm a Python developer, but work still forces me to take design pattern courses written for Java…

In [19]:
class Borg:
    __shared = {}
    
    def __init__(self):
        self.__dict__ = self.__shared
    
    def __eq__(self, other) -> bool:
        """Implement equivalence testing by comparing the instance attribute stores."""
        
        if not isinstance(other, Borg): return False
        return self.__dict__ is getattr(other, '__dict__', None)
        # We're careful, above, as not every object has a __dict__ attribute.

Hold up a minute, here.  This looks kinda weird.  Several things have been introduced at the same time, which is not usually very helpful for learning, so let's break it down.

We create a class to do what we want. Standard code encapsulation technique. Check.

This class has a class-scoped dictionary attribute named `__shared` — this is where the _data_ associated with each instance will be pooled. As can be seen within the constructor, we _re_-assign the `__dict__` attribute of the instance we are initializing, and swap it out for the shared dictionary instance.  This `__dict__` attribute is where attributes associated with only the instance (vs. shared and passed down from the class) are stored.

Executing this, this doesn't *quite* behave like a singleton, as they're not singular:

In [20]:
a = Borg()
b = Borg()

a is b

False

However, they are equal:

In [21]:
a == b

True

And attributes assigned to one propagate to the other:

In [22]:
a.foo = 27
b.foo

27